In [1]:
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [2]:
X, y = make_classification(n_samples=10000, n_features=10, n_informative=3)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
dt = DecisionTreeClassifier(random_state=42)

dt.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [5]:
y_pred = dt.predict(X_test)

print("Decision Tree Accuracy : ", accuracy_score(y_test, y_pred))

Decision Tree Accuracy :  0.916


## Bagging

In [6]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    random_state=42
)

In [7]:
bag.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.25,
                  n_estimators=500, random_state=42)

In [8]:
y_pred = bag.predict(X_test)

In [9]:
accuracy_score(y_test, y_pred)

0.9375

In [10]:
bag.estimators_samples_[0].shape

(2000,)

## Bagging using SVM 

In [11]:
bag = BaggingClassifier(
    estimator=SVC(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    random_state=42
) 

In [12]:
bag.fit(X_train, y_train)

BaggingClassifier(estimator=SVC(), max_samples=0.25, n_estimators=500,
                  random_state=42)

In [13]:
y_pred = bag.predict(X_test)


In [14]:

print("Accuracy score of SVC : ", accuracy_score(y_test, y_pred))

Accuracy score of SVC :  0.9225


## Pasting 

In [26]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=False,    # => sampling without replacement
    random_state=42,
    verbose=1,
    n_jobs=-1
) 

In [27]:
bag.fit(X_train, y_train)

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   2 out of  12 | elapsed:    2.0s remaining:   10.2s
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:    2.2s finished


BaggingClassifier(bootstrap=False, estimator=DecisionTreeClassifier(),
                  max_samples=0.25, n_estimators=500, n_jobs=-1,
                  random_state=42, verbose=1)

In [28]:
y_pred = bag.predict(X_test)

print("Pasting classifier",accuracy_score(y_test,y_pred))

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   2 out of  12 | elapsed:    0.0s remaining:    0.4s


Pasting classifier 0.9375


[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:    0.2s finished


## Random subspaces

In [23]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=1.0,
    bootstrap=False,    # => row sampling without replacement
    
    max_features=0.5,
    bootstrap_features=False,    # => column sampling without replacement
    random_state=42
) 

In [24]:
bag.fit(X_train, y_train)

BaggingClassifier(bootstrap=False, estimator=DecisionTreeClassifier(),
                  max_features=0.5, n_estimators=500, random_state=42)

In [25]:
y_pred = bag.predict(X_test)

print("Random subspaces classifier: ", accuracy_score(y_test, y_pred))


Random subspaces classifier:  0.9375


## Random Patches

In [30]:

bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)

In [31]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Random Patches classifier",accuracy_score(y_test,y_pred))

Random Patches classifier 0.9345


## OOB Score

In [33]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    oob_score=True,
    random_state=42
)
     


In [34]:
bag.fit(X_train,y_train)



BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.25,
                  n_estimators=500, oob_score=True, random_state=42)

In [36]:

bag.oob_score_

0.945625

In [37]:

y_pred = bag.predict(X_test)
print("Accuracy",accuracy_score(y_test,y_pred))

Accuracy 0.9375


## Bagging Tips

Bagging generally gives better results than Pasting

Good results come around the 25% to 50% row sampling mark

Random patches and subspaces should be used while dealing with high dimensional data

To find the correct hyperparameter values we can do GridSearchCV/RandomSearchCV

## Applying GridSearchCV

In [38]:

from sklearn.model_selection import GridSearchCV

In [39]:
parameters = {
    'n_estimators': [50,100,500], 
    'max_samples': [0.1,0.4,0.7,1.0],
    'bootstrap' : [True,False],
    'max_features' : [0.1,0.4,0.7,1.0]
    }

In [40]:

search = GridSearchCV(BaggingClassifier(), parameters, cv=5)

In [ ]:
search.fit(X_train,y_train)


In [ ]:

print(search.best_params_)
print(search.best_score_)

In [ ]:


search.best_params_